In [1]:
import os
import numpy as np
from skimage import io, measure
from skimage.measure import regionprops
import glob
from ultralytics import YOLO

import torch
import napari
import skimage as ski
import pandas as pd
import plotly.express as px

WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at 'C:\Users\smc\AppData\Roaming\Ultralytics\settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
viewer = napari.Viewer()

# Check Results

In [3]:
fnames = glob.glob('runs/detect/*/results.csv')
fnames

['runs/detect\\train12\\results.csv',
 'runs/detect\\train16\\results.csv',
 'runs/detect\\train18\\results.csv',
 'runs/detect\\train19\\results.csv',
 'runs/detect\\train20\\results.csv',
 'runs/detect\\train21\\results.csv',
 'runs/detect\\train22\\results.csv',
 'runs/detect\\train23\\results.csv',
 'runs/detect\\train24\\results.csv',
 'runs/detect\\train25\\results.csv',
 'runs/detect\\train26\\results.csv',
 'runs/detect\\train27\\results.csv',
 'runs/detect\\train28\\results.csv',
 'runs/detect\\train29\\results.csv',
 'runs/detect\\train30\\results.csv',
 'runs/detect\\train33\\results.csv',
 'runs/detect\\train34\\results.csv',
 'runs/detect\\train35\\results.csv',
 'runs/detect\\train36\\results.csv',
 'runs/detect\\train37\\results.csv',
 'runs/detect\\train38\\results.csv',
 'runs/detect\\train6\\results.csv',
 'runs/detect\\train7\\results.csv',
 'runs/detect\\train8\\results.csv']

In [4]:
df = pd.DataFrame()
for fname in fnames:
    cdf = pd.read_csv(fname)
    cdf['file'] = fname
    df = pd.concat([df, cdf])

In [5]:
cls_loss = df.columns[9]
epoch = df.columns[0]

In [6]:
# This has not been very useful

px.line(df, x=epoch, y=cls_loss, color='file')

# Batching

In [7]:
variant = '36'

In [8]:
model = YOLO('runs/detect/train'+variant+'/weights/best.pt')  # load a custom model

In [9]:
fnames = glob.glob('S:/micro/nro/fx2482/lem/smc/20240820_OSS_IMARE-130719/*.tif')

folders = ['S:/micro/nro/fx2482/lem/smc/20240712_OSS_IMARE-129759/',
'S:/micro/nro/fx2482/lem/smc/20240715_OSS_IMARE-129791/',
'S:/micro/nro/fx2482/lem/smc/20240716_OSS_IMARE-129813_2/',
'S:/micro/nro/fx2482/lem/smc/20240717_OSS_IMARE-129848/',
'S:/micro/nro/fx2482/lem/smc/20240722_OSS_IMARE-129920/',
'S:/micro/nro/fx2482/lem/smc/20240726_OSS_IMARE-130092/',
'S:/micro/nro/fx2482/lem/smc/20240729_OSS_IMARE-130129/',
'S:/micro/nro/fx2482/lem/smc/20240729_OSS_IMARE-130129-2/',
]
fnames = []
for folder in folders:
    fnames += glob.glob(folder+'*.tif')
fnames.sort()

folders = ['S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379/',
           'S:/micro/nro/fx2482/lem/20240924_OSS_IMARE-131458/']
fnames = []
for folder in folders:
    fnames += glob.glob(folder+'*.tif')
fnames.sort()


In [10]:
def pad_to_window(img, window=10000):
    height, width = img.shape[0:2]
    new_img = np.pad(img, ((0, window-height), (0, window-width), (0,0)), mode='constant', constant_values=255)
    return new_img

In [11]:
def window_images(img, window=10000):
    shape = img.shape[0:2]
    y_comb = []
    for y in range(0, shape[0], window):
        x_comb = []
        for x in range(0, shape[1], window):
            x_comb.append(pad_to_window(img[y:y+window, x:x+window]))
        y_comb.append(x_comb)
    return np.array(y_comb)


In [12]:
def de_window(img):
    rtn_img = np.concatenate([np.concatenate([x for x in simg], axis=1) for simg in img], axis=0)
    return rtn_img

In [13]:
def process_file(fname, display=False, viewer=None, confidence_cutoff=0.5):
    img = ski.io.imread(fname)
    windowed = window_images(img)
    confs = []
    rtn_boxes = []
    classes = []
    for y,yimg in enumerate(windowed):
        for x,ximg in enumerate(yimg):
            height = 10000
            width = 10000
            results = model(ximg)
            for box in results[0].boxes:
                xyxy = np.reshape(box.xyxy[0].cpu().numpy(), [-1,2])[:,::-1]
                xyxy = xyxy + np.array([y*10000, x*10000])[np.newaxis,:]
                rtn_boxes.append(xyxy)
                classes.append(box.cls.cpu().numpy()[0])
                confs.append(box.conf[0].cpu().numpy())
    classes = np.array(classes)
    rtn_boxes = np.array(rtn_boxes)
    confs = np.array(confs)

    classes = classes[confs > confidence_cutoff]
    rtn_boxes = rtn_boxes[confs > confidence_cutoff]
    confs = confs[confs > confidence_cutoff]
    rtn_img = de_window(windowed)

    if display:
        viewer.layers.clear()
        viewer.add_image(rtn_img, channel_axis=-1, colormap=['red', 'green', 'blue'])
        viewer.add_shapes(rtn_boxes[classes==0], shape_type='rectangle', edge_color='red', face_color='red', opacity=0.5, name='T0')
        viewer.add_shapes(rtn_boxes[classes==1], shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.5, name='T1')
        viewer.add_shapes(rtn_boxes[classes==2], shape_type='rectangle', edge_color='green', face_color='green', opacity=0.5, name='T2')
        viewer.add_shapes(rtn_boxes[classes==3], shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.5, name='T3')

    return rtn_img, rtn_boxes, classes, confs



In [14]:
fname = fnames[15]
#fname = 'S:/micro/nro/fx2482/lem/smc/20240712_OSS_IMARE-129759/01_16_07.tif'
print(fname)
img, boxes, classes, confs = process_file(fname, display=True, viewer=viewer, confidence_cutoff=0.0)

S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379\01_04_04.tif

0: 1600x1600 122 1_stage_IBs, 42 2_stage_IIs, 72 4_stage_late_IIIs, 39.0ms
Speed: 30.0ms preprocess, 39.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1600, 1600)


In [15]:
df = pd.DataFrame()
for fname in fnames:
    print(fname)
    img, boxes, classes, confs = process_file(fname)
    if len(classes)>1:
        areas = np.prod(boxes[:,1] - boxes[:,0], axis=1)
        cdf = pd.DataFrame({'file': fname, 'class': classes, 'conf': confs, 'area':areas})
    df = pd.concat([df, cdf])

S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379\01_01_02.tif

0: 1600x1600 102 1_stage_IBs, 30 2_stage_IIs, 59 4_stage_late_IIIs, 44.0ms
Speed: 254.0ms preprocess, 44.0ms inference, 9.0ms postprocess per image at shape (1, 3, 1600, 1600)
S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379\01_01_03.tif

0: 1600x1600 112 1_stage_IBs, 26 2_stage_IIs, 70 4_stage_late_IIIs, 39.0ms
Speed: 27.0ms preprocess, 39.0ms inference, 11.0ms postprocess per image at shape (1, 3, 1600, 1600)
S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379\01_01_04.tif

0: 1600x1600 107 1_stage_IBs, 29 2_stage_IIs, 69 4_stage_late_IIIs, 96.0ms
Speed: 28.0ms preprocess, 96.0ms inference, 12.0ms postprocess per image at shape (1, 3, 1600, 1600)
S:/micro/nro/fx2482/lem/20240923_OSS_IMARE-131379\01_01_05.tif

0: 1600x1600 108 1_stage_IBs, 27 2_stage_IIs, 72 4_stage_late_IIIs, 75.0ms
Speed: 27.0ms preprocess, 75.0ms inference, 12.0ms postprocess per image at shape (1, 3, 1600, 1600)
S:/micro/nro/fx2482/lem/20240923_OSS_IMA

In [13]:
# Took 41m to run through all the files

In [16]:
df.to_csv('S:/micro/nro/fx2482/lem/smc/20240923_'+variant+'.csv')

viewer2.add_image(img, channel_axis=-1, colormap=['red', 'green', 'blue'])

viewer2.add_shapes(viewer.layers[-4].data, shape_type='rectangle', edge_color='red', face_color='red', opacity=0.5, name='T0')
viewer2.add_shapes(viewer.layers[-3].data, shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.5, name='T1')
viewer2.add_shapes(viewer.layers[-2].data, shape_type='rectangle', edge_color='green', face_color='green', opacity=0.5, name='T2')
viewer2.add_shapes(viewer.layers[-1].data, shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.5, name='T3')

viewer = viewer2

In [43]:
df.shape

(278138, 4)